In [1]:
import os
import pandas as pd
import jieba
import numpy as np
from collections import Counter
import pickle
from tqdm import tqdm

D:\Anaconda\envs\dl\lib\site-packages\jieba\_compat.py:18: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [8]:
train_df=pd.read_csv('../data/processed/train.csv')
test_df=pd.read_csv('../data/processed/test.csv')
val_df=pd.read_csv('../data/processed/val.csv')
print(f'训练集{train_df.shape}')
print(f'测试集{test_df.shape}')
print(f'验证集{val_df.shape}')
print(f"\n类别: {train_df['label'].unique()}")

训练集(44352, 3)
测试集(9504, 3)
验证集(9504, 3)

类别: [13  9 11  8  1 10  3 12  7  4 14  5  6  2  0]


In [9]:
# 中文停用词表
stopwords = set([
    '的', '了', '在', '是', '我', '有', '和', '就', '不', '人', '都',
    '一', '一个', '上', '也', '很', '到', '说', '要', '去', '你', '会',
    '着', '没有', '看', '好', '自己', '这', '中', '以', '来', '个',
    '地', '为', '他', '得', '她', '对', '么', '里', '后', '能', '再',
    '而', '被', '从', '把', '让', '与', '等', '别', '之', '这个',
])

print(f"停用词数量: {len(stopwords)}")

停用词数量: 53


In [10]:
def tokenize(text):
    words=jieba.cut(text)
    words=[w for w in words if w not in stopwords]
    return words
# 测试分词
sample_text = train_df.iloc[0]['text']
print("原文:")
print(sample_text[:100])
print("\n分词后:")
print(" / ".join(tokenize(sample_text)[:20]))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ysn\AppData\Local\Temp\jieba.cache


原文:
玉兰花开红艳艳

分词后:


Loading model cost 0.892 seconds.
Prefix dict has been built successfully.


玉兰花 / 开 / 红艳艳


In [12]:
# 对所有数据分词
from tqdm import tqdm

# 重要！先启用pandas的进度条
tqdm.pandas()
train_df['words']=train_df['text'].progress_apply(tokenize)
test_df['words']=test_df['text'].progress_apply(tokenize)
val_df['words']=val_df['text'].progress_apply(tokenize)

print("\n 分词完成！")

# 查看分词后的结果
print("\n示例（前3条）:")
for i in range(3):
    print(f"\n第{i+1}条:")
    print(f"标签: {train_df.iloc[i]['label']}")
    print(f"分词: {' '.join(train_df.iloc[i]['words'][:15])}...")

100%|██████████| 9504/9504 [00:02<00:00, 3600.04it/s]


 分词完成！

示例（前3条）:

第1条:
标签: 13
分词: 玉兰花 开 红艳艳...

第2条:
标签: 9
分词: 建造 一艘 航母 多难 ？ 飞行 甲板 出来 ， 知道 有多厚 吗 ？...

第3条:
标签: 11
分词: 河南 ， 湖南 ， 海南 谁 世界 知名度 高 ？...


In [14]:
all_words=[]
for word in train_df['words']:
    all_words.extend(word)
    
word_freq=Counter(all_words)
print(f"总词: {len(all_words)}")
print(f"不重复词数:{word_freq}")

min_freq=5
word_freq={word:freq for word, freq in word_freq.items() if freq>=min_freq}
print(f"保留词数: {len(word_freq)}")

sorted_words=sorted(word_freq.items(), key=lambda x:x[1], reverse=True)
# 构建词到ID的映射
vocab = {'<PAD>': 0, '<UNK>': 1}  # 特殊token
for word, freq in sorted_words:
    vocab[word] = len(vocab)

print(f"\n词表大小: {len(vocab):,}")

print(f"\nTop 20 高频词:")
for i, (word, freq) in enumerate(sorted_words[:20], 1):
    print(f"{i:2d}. {word}: {freq:,}")

# 保存词表
with open('../data/processed/vocab.pkl', 'wb') as f:
    pickle.dump(vocab, f)

print("\n 词表已保存到 data/processed/vocab.pkl")

总词: 491966
不重复词数:Counter({'，': 28946, '？': 20407, '！': 7881, '：': 6308, ' ': 6011, '“': 4644, '”': 4633, '吗': 3275, '什么': 2597, '中国': 2404, '为什么': 2317, '如何': 2277, '怎么': 1967, '《': 1760, '》': 1760, '、': 1521, '美国': 1392, '年': 1306, '最': 1212, '—': 1172, '还': 1067, '将': 1060, '大': 1052, '哪些': 995, '5': 886, '世界': 865, '2018': 828, '却': 818, '谁': 812, '月': 812, '新': 801, '可以': 800, '多': 772, '又': 765, '网友': 726, '做': 714, '还是': 709, '上联': 705, '日本': 695, '用': 693, '万': 687, '如果': 683, '下联': 682, '手机': 663, '农村': 661, '现在': 653, '买': 644, '…': 638, '为何': 622, '「': 589, '」': 589, '呢': 580, '怎样': 579, '-': 564, '给': 558, '岁': 558, '3': 545, '｜': 532, '10': 524, '多少': 523, '国家': 520, '到底': 515, '知道': 512, '）': 509, '（': 506, '哪个': 501, '俄罗斯': 497, '真的': 489, '想': 477, '4': 476, '更': 475, '日': 470, '我们': 468, '这些': 466, '过': 466, '小': 459, '游戏': 457, '下': 453, '看待': 435, '怎么样': 434, '2': 432, '钱': 415, '成为': 410, '王者': 408, '城市': 402, '这么': 399, '孩子': 395, '发展': 391, '荣耀': 390, '没': 388, '汽车

In [15]:
# Cell 7: 文本转ID序列
print("将文本转换为ID序列...")

def words_to_ids(words, vocab, max_len=512):
    """将词列表转换为ID列表"""
    ids = [vocab.get(word, vocab['<UNK>']) for word in words]

    if len(ids) > max_len:
        ids = ids[:max_len]
    else:
        ids = ids + [vocab['<PAD>']] * (max_len - len(ids))

    return ids

# 转换
train_df['ids'] = train_df['words'].apply(lambda x: words_to_ids(x, vocab))
val_df['ids'] = val_df['words'].apply(lambda x: words_to_ids(x, vocab))
test_df['ids'] = test_df['words'].apply(lambda x: words_to_ids(x, vocab))

print("ID转换完成！")

# 示例
idx = 0
print(f"\n示例:")
print(f"原文: {train_df.iloc[idx]['text'][:50]}...")
print(f"分词: {' '.join(train_df.iloc[idx]['words'][:10])}...")
print(f"ID序列前20个: {train_df.iloc[idx]['ids'][:20]}")

将文本转换为ID序列...
ID转换完成！

示例:
原文: 玉兰花开红艳艳...
分词: 玉兰花 开 红艳艳...
ID序列前20个: [1, 171, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [16]:
# Cell 8: 保存标签映射
import pickle

label_map_save = {
    0: '故事', 1: '文化', 2: '娱乐', 3: '体育', 4: '财经',
    5: '房产', 6: '汽车', 7: '教育', 8: '科技', 9: '军事',
    10: '旅游', 11: '国际', 12: '股票', 13: '农业', 14: '游戏'
}

with open('../data/processed/label_map.pkl', 'wb') as f:
    pickle.dump(label_map_save, f)

print("标签映射已保存")
print("\n标签映射:")
for k, v in label_map_save.items():
    print(f"{k:2d} → {v}")

标签映射已保存

标签映射:
 0 → 故事
 1 → 文化
 2 → 娱乐
 3 → 体育
 4 → 财经
 5 → 房产
 6 → 汽车
 7 → 教育
 8 → 科技
 9 → 军事
10 → 旅游
11 → 国际
12 → 股票
13 → 农业
14 → 游戏


In [17]:
# Cell 9: 保存最终数据并总结
import pickle

print("保存最终处理结果...")

# 保存完整数据（包含text, label, label_name, words, ids等）
train_df.to_pickle('../data/processed/train_processed.pkl')
val_df.to_pickle('../data/processed/val_processed.pkl')
test_df.to_pickle('../data/processed/test_processed.pkl')

print("\n" + "="*60)
print("🎉 数据预处理全部完成！")
print("="*60)

print("\n数据统计:")
print(f"  训练集: {len(train_df):,} 条")
print(f"  验证集: {len(val_df):,} 条")
print(f"  测试集: {len(test_df):,} 条 （有label，可以评估）")
print(f"  词表大小: {len(vocab):,}")
print(f"  类别数: 15")
print(f"  序列长度: 512")


保存最终处理结果...

🎉 数据预处理全部完成！

数据统计:
  训练集: 44,352 条
  验证集: 9,504 条
  测试集: 9,504 条 （有label，可以评估）
  词表大小: 11,871
  类别数: 15
  序列长度: 512
